In [ ]:
# Importing the required libraries
import ee
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

## Authenticate and Initialize Earth Engine

To access and use the Google Earth Engine (GEE) API, we need to authenticate and initialize it. The authentication process will prompt you to log in to your Google account and grant the necessary permissions for Earth Engine. Once authenticated, we initialize the Earth Engine with a specific project name.

The following code performs these steps:

In [ ]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='ee-dharanideivasihamani')

## Function: Compute Water Extent Time Series

The `compute_water_extent_time_series` function is designed to analyze the changes in water extent over a specified period using the JRC Global Surface Water dataset. This function reads a GeoJSON file to define the geometry of a water body and calculates the surface water area for each month within the specified date range. The results are returned as a time series DataFrame.

### Parameters
- `geojson_path` (str): The file path to the GeoJSON file containing the geometry of the water body.
- `start_date` (str): The start date for the analysis period in the format 'YYYY-MM-DD'.
- `end_date` (str): The end date for the analysis period in the format 'YYYY-MM-DD'.

### Returns
- `df` (pandas.DataFrame): A DataFrame containing the date and the corresponding surface water area in square meters.

### Function Steps
1. **Read GeoJSON File**: The function reads the GeoJSON file to extract the geometry of the water body.
2. **Convert to Earth Engine Geometry**: The extracted geometry is converted into an Earth Engine polygon.
3. **Define Time Range**: The start and end dates are converted into Earth Engine date objects.
4. **Load Dataset**: The JRC Global Surface Water dataset is loaded and filtered based on the date range and the water body geometry.
5. **Compute Water Area**: For each image in the filtered dataset, the surface water area is computed by masking non-water pixels and summing the area of water pixels.
6. **Create Time Series DataFrame**: The dates and corresponding water areas are collected into a DataFrame, which is then sorted by date and returned.


In [ ]:
# Function to compute water extent time series
def compute_water_extent_time_series(geojson_path, start_date, end_date):
    # Read GeoJSON file
    gdf = gpd.read_file(geojson_path)
    # The column no 3 refers to 4th lake. Please change the no between 0 and 3 for choosing a different lake.
    water_body_geom = gdf.geometry.iloc[3] 
    water_body_ee = ee.Geometry.Polygon(list(water_body_geom.exterior.coords))

    # Define time range
    start_date = ee.Date(start_date)
    end_date = ee.Date(end_date)

    # Load JRC Global Surface Water dataset
    dataset = ee.ImageCollection('JRC/GSW1_2/MonthlyHistory') \
                .filterDate(start_date, end_date) \
                .filterBounds(water_body_ee)

    # Function to compute water area
    def compute_water_area(image):
        water = image.select('water').gt(0).selfMask()
        water_area = water.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=water_body_ee,
            scale=10,
            maxPixels=1e9
        ).get('water')
        return water_area

    # Compute water area for each image in the dataset
    dates = []
    areas = []
    for image in dataset.toList(dataset.size()).getInfo():
        date = image['properties']['system:time_start']
        date = pd.to_datetime(date, unit='ms').strftime('%Y-%m-%d')
        water_image = ee.Image(image['id'])
        area = compute_water_area(water_image).getInfo()
        dates.append(date)
        areas.append(area if area is not None else 0)

    # Create time series DataFrame
    df = pd.DataFrame({'date': pd.to_datetime(dates), 'area_sqm': areas})
    df = df.sort_values(by='date')

    return df

In [ ]:
# Specify GeoJSON file path, time range, and output image path
geojson_path = 'D:\\NatureDots\\data\\ne_10m_lakes_historic.geojson' # Local file path, please change accordingly
start_date = '2016-01-01'
end_date = '2023-01-01'

In [ ]:
# Compute water extent time series
df = compute_water_extent_time_series(geojson_path, start_date, end_date)

## Plot Time Series

The following code snippet is used to visualize the changes in surface water extent over time by plotting the time series data contained in the DataFrame generated by the `compute_water_extent_time_series` function. The plot displays the surface water area in square meters against the corresponding dates. 

In [ ]:
# Plot time series
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['area_sqm'], marker='o')
plt.xlabel('Date')
plt.ylabel('Surface Water Area (sqm)')
plt.title('Surface Water Extent Over Time')
plt.grid(True)
plt.savefig('surface_water_extent_time_series_10m_lake_3.png')
plt.show()